## Can we make use of the LLL algorithm in finance?

Maybe?

Suppose we have a set of financial instruments whose risk can be represented as a vector in a high-dimensional space. We want to come up with a portfolio with low cardinality that has minimal risk. We can construct a lattice whose basis vectors represent the risk of each instrument. The LLL algorithm can then be used to find a short vector (low risk) in this lattice that is a linear combination of the basis vectors. The coefficients in the linear combination can be interpreted as positions for each instrument in the portfolio.

Here, we use 3-year data for the current S&P 500 stocks to construct risk vectors for each stock. The four risk metrics that we have chosen are beta, volatility, Amihud illiquidity (returns divided by volume), and tail risk (correlation with extreme movements in the index).

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from fpylll import IntegerMatrix, LLL

In [2]:
def fetch_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    data = data.dropna(axis=1, how='all')  # Drop columns with all NaN values
    data = data.dropna(axis=0, how='all')  # Drop rows with all NaN values
    data = data.ffill()  # Fill NaN values with the previous valid observation
    return data

In [3]:
def standardise_risk_matrix(raw_risk_matrix):
    risk_matrix = raw_risk_matrix.copy()
    # Min-max scale Volatility and Amihud Illiquidity to [0, 1]
    for col in ['Volatility', 'Amihud Illiquidity']:
        if col in risk_matrix.columns:
            min_val = risk_matrix[col].min()
            max_val = risk_matrix[col].max()
            risk_matrix[col] = (risk_matrix[col] - min_val) / (max_val - min_val)

    return risk_matrix

In [4]:
def calculate_risk_matrix(data):
    close_data = data['Close'].dropna(axis=1, how='any')
    volume_data = data['Volume'].dropna(axis=1, how='any')
    volume_data = volume_data.replace(0, np.nan).ffill()
    returns = close_data.pct_change().dropna()  # Calculate daily returns
    sp500_returns = returns['^GSPC']  # S&P 500 returns
    betas = returns.corrwith(sp500_returns) # Beta of each stock
    volatilities = returns.std()  # Volatility of each stock
    amihud_illiquidity = ((1e6)*(np.abs(returns)/volume_data).mean())  # Amihud Illiquidity Measure
    left_threshold = returns['^GSPC'].quantile(0.05)
    right_threshold = returns['^GSPC'].quantile(0.95)
    tail_data = returns[(returns['^GSPC'] >= right_threshold) | (returns['^GSPC'] <= left_threshold)]
    tail_corr = tail_data.corr()
    tail_risk = tail_corr['^GSPC']
    raw_risk_matrix = pd.DataFrame({
        'Beta': betas,
        'Volatility': volatilities,
        'Amihud Illiquidity': amihud_illiquidity,
        'Tail Risk': tail_risk
    })
    # standardize the risk matrix using min-max scaling
    risk_matrix = standardise_risk_matrix(raw_risk_matrix)
    # drop GSPC row
    risk_matrix = risk_matrix.drop(index='^GSPC', errors='ignore')
    # shuffle order of rows
    risk_matrix = risk_matrix.sample(frac=1, random_state=42)

    return risk_matrix

In [ ]:
def find_stock_combinations_LLL(risk_matrix):
    # Scale to integers if needed (LLL requires integer matrix)
    scale = 1e6
    basis = np.round(risk_matrix.values * scale).astype(int)
    n_rows, n_cols = basis.shape
    np.random.seed(42)  # For reproducibility
    pad = np.random.randint(0, 2, size=(n_rows, n_rows - n_cols)) # Random padding with 0s and 1s to make it a square matrix
    basis = np.hstack([basis, pad])
    R = IntegerMatrix.from_matrix(basis)
    U = IntegerMatrix.identity(R.nrows)
    LLL.reduction(R, U)
    z = np.zeros((risk_matrix.shape[0], risk_matrix.shape[0]), dtype=int)
    _ = U.to_matrix(z)
    # create a dictionary to map each row index to the corresponding stock combinations and the quantity
    stock_combinations = {
        i:  {
            risk_matrix.index[j]: z[i, j] for j in range(len(z[i])) if z[i, j] != 0
        }
        for i in range(z.shape[0])
    }
    return stock_combinations

In [6]:
# # Scrape S&P 500 tickers from Wikipedia
# url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
# table = pd.read_html(url)
# sp500 = table[0]
# tickers = sp500['Symbol'].tolist()

# # convert tickers to yfinance format
# tickers = [ticker.replace('.', '-') for ticker in tickers]
# tickers = ['^GSPC'] + tickers  # Add S&P 500 index ticker

# # Fetch stock data for the S&P 500 companies
# start_date = '2020-01-01'
# end_date = '2023-01-01'
# data = fetch_stock_data(tickers, start_date, end_date)

data = pd.read_pickle('sp500_data.pkl')

In [7]:
risk_matrix = calculate_risk_matrix(data)

In [8]:
stock_combinations_LLL = find_stock_combinations_LLL(risk_matrix)

### Convex optimisation

For the sake of comparison, we also construct a portfolio (using similar principles/constraints) using convex optimisation. Depending upon the constraints, the convex optimisation routine can take anywhere from a few seconds to a few minutes to run (sometimes returning infinity), while the LLL algorithm always finds a solution in a few seconds. As shown below, both solutions are comparable in finding a portfolio with low cardinality and negligible risk.

In [9]:
def find_stock_combination_cvx(risk_matrix, max_position=10, min_position=1):
    n_stocks = len(risk_matrix)
    positions = cp.Variable(n_stocks)
    y_pos = cp.Variable(n_stocks, boolean=True)
    y_neg = cp.Variable(n_stocks, boolean=True)

    # Create risk matrix as numpy array
    R = risk_matrix.values

    # Objective: Minimize Euclidean norm of risk vector
    objective = cp.Minimize(cp.norm(R.T @ positions, 2))

    # Constraints
    constraints = [
        cp.sum(y_pos) >= 1,
        cp.sum(y_neg) >= 1,
    ]

    for i in range(n_stocks):
        constraints.append(y_pos[i] + y_neg[i] <= 1)  # Only one of pos/neg per stock

        constraints.append(positions[i] >= min_position * y_pos[i] - max_position * (1 - y_pos[i]))
        constraints.append(positions[i] <= max_position * y_pos[i])

        constraints.append(positions[i] <= -min_position * y_neg[i] + max_position * (1 - y_neg[i]))
        constraints.append(positions[i] >= -max_position * y_neg[i])

    # Solve problem
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS_BB)
    print(f"CVX portfolio risk: {prob.value}")

    stock_combination = {
        risk_matrix.index[i]: np.round(positions.value[i]) for i in range(n_stocks)
        if np.round(positions.value[i]) != 0
    }
   
    return stock_combination

In [10]:
stock_combination_cvx = find_stock_combination_cvx(risk_matrix)

CVX portfolio risk: 4.1284935527842153e-13


In [14]:
LLL_portfolio_risks = [
    np.linalg.norm(
        np.sum([risk_matrix.loc[s]*stock_combinations_LLL[i][s] for s in stock_combinations_LLL[i].keys()])
    )
    for i in range(len(stock_combinations_LLL))
]

print(f"All LLL portfolios have risk lower than {np.max(LLL_portfolio_risks)}")

All LLL portfolios have risk lower than 7.641257292867465e-05


In [12]:
stock_combination_cvx

{'FICO': np.float64(3.0),
 'TYL': np.float64(-2.0),
 'TPL': np.float64(-3.0),
 'NVR': np.float64(-1.0),
 'ERIE': np.float64(10.0),
 'AZO': np.float64(-5.0)}

In [13]:
# find the LLL stock combination with the fewest stocks
min_stocks = min(len(comb) for comb in stock_combinations_LLL.values())
min_combination = {k: v for k, v in stock_combinations_LLL.items() if len(v) == min_stocks}
min_combination

{344: {'URI': np.int64(-1),
  'CHD': np.int64(-1),
  'TRV': np.int64(1),
  'WFC': np.int64(1),
  'TMUS': np.int64(1),
  'TER': np.int64(-1),
  'BXP': np.int64(1),
  'TPR': np.int64(-1),
  'HD': np.int64(1),
  'CCI': np.int64(-1),
  'FICO': np.int64(1),
  'EMR': np.int64(-1),
  'AON': np.int64(1),
  'IDXX': np.int64(-1),
  'SBAC': np.int64(-1),
  'PARA': np.int64(1)}}

### Limitations:
1. The financial instruments might be correlated which is not taken into account here.
2. To make the matrix full-rank, we have to add some noise to the risk vectors, otherwise it gives a zero vector.
3. It is more likely to pick from among the first few instruments.
4. Risk vectors for long positions should look different from those for short positions, which is not considered here.

### Advantages of the lattice reduction approach:
1. The LLL algorithm is efficient and can handle high-dimensional spaces quite comfortably.
2. It can be used to find multiple candidate portfolios with low cardinality with minor tweaks in the setup.
3. Unlike convex optimisation, the objective function (the norm of the short vector here) can be non-convex, which might allow for more flexibility in how the risk (basis) vectors are constructed.